# Object detection using Transfer Learning of CNN architectures



1.   Load in a pre-trained CNN model trained on a large dataset
2.   Freeze parameters(weights) in the model’s lower convolutional layers
3.   Add a custom classifier with several layers of trainable parameters to model
4.   Train classifier layers on training data available for the task
5.   Fine-tune hyperparameters and unfreeze more layers as needed




In [ ]:
from keras.applications import ResNet50
from keras.layers import Flatten, Dense, Dropout
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_path = '/content/drive/MyDrive/DL 6/train'
valid_path = '/content/drive/MyDrive/DL 6/val'

In [ ]:
# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 [==============================] - 3s 0us/step


In [ ]:
# Freeze early layers
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Add custom classifier
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)

In [ ]:
# Create full model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
# Create generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(224, 224),
        batch_size=2,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        valid_path,
        target_size=(224, 224),
        batch_size=2,
        class_mode='categorical')

Found 72 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [ ]:
# Train classifier layers
model.fit(
        train_generator,
        steps_per_epoch=5,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=50)

Epoch 1/10
5/5 [==============================] - ETA: 0s - loss: 66.4406 - accuracy: 0.8000

5/5 [==============================] - 29s 4s/step - loss: 66.4406 - accuracy: 0.8000 - val_loss: 57.5446 - val_accuracy: 0.5000
Epoch 2/10
5/5 [==============================] - 4s 694ms/step - loss: 34.3868 - accuracy: 0.5000
Epoch 3/10
5/5 [==============================] - 3s 705ms/step - loss: 13.7933 - accuracy: 0.6000
Epoch 4/10
5/5 [==============================] - 3s 403ms/step - loss: 25.2056 - accuracy: 0.4000
Epoch 5/10
5/5 [==============================] - 3s 470ms/step - loss: 12.9994 - accuracy: 0.7000
Epoch 6/10
5/5 [==============================] - 2s 432ms/step - loss: 14.9081 - accuracy: 0.4000
Epoch 7/10
5/5 [==============================] - 1s 317ms/step - loss: 20.2821 - accuracy: 0.3000
Epoch 8/10
5/5 [==============================] - 1s 313ms/step - loss: 7.1835 - accuracy: 0.6000
Epoch 9/10
5/5 [==============================] - 1s 176ms/step - loss: 13.8812 - accuracy: 0.5000
Epoch 10/10
5/5 [==============================] - 2s 401ms/step - loss: 9.6148 

In [ ]:
# Unfreeze deeper layers
for layer in base_model.layers[-6:]:
    layer.trainable = True

# Recompile and fine-tune model
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(
        train_generator,
        steps_per_epoch=5,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=50)

Epoch 1/10
5/5 [==============================] - ETA: 0s - loss: 1432.2275 - accuracy: 0.3000

5/5 [==============================] - 6s 496ms/step - loss: 1432.2275 - accuracy: 0.3000 - val_loss: 9025.9902 - val_accuracy: 0.5000
Epoch 2/10
5/5 [==============================] - 0s 81ms/step - loss: 11576.0137 - accuracy: 0.7000
Epoch 3/10
5/5 [==============================] - 1s 169ms/step - loss: 2737.4167 - accuracy: 0.6000
Epoch 4/10
5/5 [==============================] - 0s 76ms/step - loss: 1798757.6250 - accuracy: 0.5000
Epoch 5/10
5/5 [==============================] - 0s 76ms/step - loss: 2559356672.0000 - accuracy: 0.6000
Epoch 6/10
5/5 [==============================] - 0s 77ms/step - loss: 12065626412785594269696.0000 - accuracy: 0.6000
Epoch 7/10
5/5 [==============================] - 0s 59ms/step - loss: nan - accuracy: 0.3000
Epoch 8/10
5/5 [==============================] - 1s 161ms/step - loss: nan - accuracy: 0.5000
Epoch 9/10
5/5 [==============================] - 0s 48ms/step - loss: nan - accuracy: 0.7000
Epoch 10/10
5/5 [==============================] - 0